# Data Loading and Cleaning

In [79]:
# imports
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np
import os
import sklearn
import string as str
import re

In [2]:
# Loading csv with preprocessed data
df = pd.read_csv('Preprocessed_data.csv')

# Cleaning the data
# Removing zero ratings
df = df[df['rating'] != 0]

In [3]:
# Removing image_url columns because we will not use it.
df = df.drop(['img_s'], axis=1)
df = df.drop(['img_m'], axis=1)
df = df.drop(['img_l'], axis=1)
df = df.dropna()

In [4]:
df.head()

,Unnamed: 0,user_id,location,age,isbn,rating,book_title,book_author,year_of_publication,publisher,Summary,Language,Category,city,state,country
1,1,8,"timmins, ontario, canada",34.7439,0002005018,5,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],timmins,ontario,canada
5,5,67544,"toronto, ontario, canada",30.0000,0002005018,8,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],toronto,ontario,canada
9,9,123629,"kingston, ontario, canada",34.7439,0002005018,9,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],kingston,ontario,canada
11,11,200273,"comber, ontario, canada",34.7439,0002005018,8,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],comber,ontario,canada
12,12,210926,"guelph, ontario, canada",34.7439,0002005018,9,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],guelph,ontario,canada


In [5]:
df.to_csv('Preprocessed_data_cleaned.csv', index=False)

In [6]:
# From now on we can work with df or load from the beggining 
# the Preprocessed_data_cleaned.csv file

-------------------------------------------------------------------
# Demographic Filtering

In [3]:
# Loading csv with book rating data
df_book_rating = pd.read_csv('BX-Book-Ratings.csv', sep=';',encoding= 'unicode_escape', engine='python')

#turns column to int
df_book_rating['Book-Rating']=[int(df_book_rating['Book-Rating'][rating].replace("'", '')) for rating in range(len(df_book_rating['Book-Rating']))]

# remove rows that have 0 as rating
df_book_rating = df_book_rating[df_book_rating['Book-Rating'] != 0]
df_book_rating=df_book_rating.reset_index(drop=True)
df_book_rating['ISBN']=[df_book_rating['ISBN'][rating].replace("'", '') for rating in range(len(df_book_rating['ISBN']))]

At this point we have cleaned and loaded the 2 dataframes that contain all the information we need to proceed.The 'Book-Ratings.csv' contains the ratings each user has appointed to books (the books are identified by ISBN). We will use this dataframe to calculate how many ratings each book has, as well as the average rating of each book. Also, we correlate these 2 values in a 'weighted rating score'. More specifically, this score takes into account both the number of ratings and the average rating. We chose the formula below, which is being used to calculate an according score for movies on IMDB website.

We will start using the weighted rating (WR) that IMDB is using, which is given as: 

Weighted Rating(WR)=(v/(v+m))*R)+((m/(v+m))*C)

where: 

* v is the number of votes per book
* m is the minimum votes required to be listed in the chart
* R is the average rating of the book
* C is the mean vote accrose the whole dataframe


In [5]:
#counts the number of ratings and puts them in to new dataframe
isbn_dict=Counter(df_book_rating['ISBN'])

df_isbn_votecount = pd.DataFrame(list(isbn_dict.items()), columns = ['ISBN', 'Vote-Count'])

The next step is to determine an appropriate value for m, the minimum votes required to be listed in the chart. We will use 90th percentile as our cutoff. In other words, for a book to feature in the charts, it must have more votes than at least 90% of the booksin the list.

In [10]:
m= df_isbn_votecount['Vote-Count'].quantile(0.9)
isbn_votecount = df_isbn_votecount[df_isbn_votecount['Vote-Count'] >= m]
isbn_votecount=isbn_votecount.reset_index(drop=True)
print('The cut-off is {} and the cleaned dataframe contains {} rows.' .format(m,isbn_votecount.shape[0]))

the cut-off is 4.0 and the cleaned dataframe contains 19918 rows.


Now we will calculate the average rating of each book.

In [12]:
#merging the dataframes
df_custom_rating= pd.merge(isbn_votecount,df_book_rating, how='inner', on=['ISBN'])

#Crearing out the user column because we don't need it
df_custom_rating= df_custom_rating.drop(columns='User-ID')

#Calculates the mean value of all the row with the same ISBN, hence the average rating which is put to the column 'Vote-Average'
df_custom_rating['Vote-Average'] = df_custom_rating['Book-Rating'].groupby(df_custom_rating['ISBN']).transform('mean')

#Deletes the duplicates and 'Book-Rating' column because we do not need them
df_custom_rating =df_custom_rating.drop_duplicates(subset='ISBN', keep='first')
df_custom_rating =df_custom_rating.drop(['Book-Rating'], axis=1)

df_custom_rating.reset_index(drop=True)

,ISBN,Vote-Count,Vote-Average
0,3257224281,4,6.750000
1,038550120X,81,7.580247
2,0060517794,30,8.000000
3,0671537458,17,7.176471
4,0679776818,21,7.476190
...,...,...,...
19913,3250600571,4,9.000000
19914,0886775809,4,8.000000
19915,0375700110,4,6.500000
19916,0451450019,4,8.000000


Above we see the generated values for the average rating of each book. Now, it is time to calculate the weighted score mentioned before. For that we need the C value, which is the mean vote across the dataframe. Then we define a function according to the formula mentioned above and then we generate the weighted score for each book.

In [13]:
C= df_custom_rating['Vote-Average'].mean()
print('The value of C is: {}'.format(C))

#Weighted score function
def weighted_rating(x, m=m, C=C):
    v = x['Vote-Count']
    R = x['Vote-Average']
    # Calculation based on the IMDB formula
    return (v/(v+m) * R) + (m/(m+v) * C) ##maybe another function for books

df_custom_rating['Score']=weighted_rating(df_custom_rating, m=m, C=C)

df_custom_rating = df_custom_rating.sort_values('Score', ascending=False)
df_custom_rating

The value of C is: 7.653260512915499


,ISBN,Vote-Count,Vote-Average,Score
144901,0439425220,23,9.869565,9.541224
141498,0618002235,25,9.720000,9.434932
175300,0060256656,20,9.750000,9.400543
211100,0836213319,13,9.923077,9.389002
63913,8826703132,21,9.714286,9.384522
...,...,...,...,...
159321,3932069145,4,3.000000,5.326630
203380,1552781631,4,3.000000,5.326630
189493,349222539X,6,3.666667,5.261304
172663,1880985055,9,3.000000,4.431772


Above we see the generated values for the weighted score. It would be nice, however, if we could see which book they are refering to without having to search with the ISBN code. For this reason we add the 'BX_Books.csv as a dataframe, which contains the titles of the books according to their ISBN, amongst other information as well. With this dataframe we can link the title of each book and display it on the above dataframe.

In [25]:
books_df=pd.read_csv('BX_Books.csv', sep=';',encoding= 'unicode_escape', engine='python')

books_df = books_df.drop(columns=["Unnamed: 5",'Unnamed: 6', "Unnamed: 7","Unnamed: 8", "Unnamed: 9"])

#cleaning the punctuation off the words 
books_df["'ISBN'"]=[books_df["'ISBN'"][i].replace("'", '') for i in range(len(books_df["'ISBN'"]))]
books_df= books_df.rename(columns={"'ISBN'": 'ISBN', "'Book-Title'": 'Book-Title'})

#merging the information we need
intersection_df = pd.merge(books_df,df_custom_rating, how='inner', on=['ISBN'])
intersection_df = intersection_df.drop(columns=["'Book-Author'","'Year-Of-Publication'","'Publisher'"])

#passing the dataframe to csv so we can work with that later
intersection_df.to_csv('Books_Votes_Score.csv', index=False)

intersection_df = intersection_df.sort_values('Vote-Count', ascending=False)
intersection_df=intersection_df.reset_index(drop=True)
intersection_df

,ISBN,Book-Title,Vote-Count,Vote-Average,Score
0,0316666343,'The Lovely Bones: A Novel',707,8.185290,8.182297
1,0971880107,'Wild Animus',581,4.390706,4.413014
2,0385504209,'The Da Vinci Code',487,8.435318,8.428947
3,0312195516,'The Red Tent (Bestselling Backlist)',383,8.182768,8.177295
4,0060928336,'Divine Secrets of the Ya-Ya Sisterhood: A Novel',320,7.887500,7.884608
...,...,...,...,...,...
18695,0486280500,'A Portrait of the Artist As a Young Man (Dove...,4,7.250000,7.451630
18696,0140421998,'Leaves of Grass (Penguin Classics)',4,7.750000,7.701630
18697,0864425295,"'Brief Encounters: stories of love, sex & travel'",4,6.500000,7.076630
18698,0395401461,'Miss Nelson Is Missing!',4,7.500000,7.576630


__By country__

In [92]:
# Loading csv with preprocessed data cleaned
df_cleaned = pd.read_csv('Preprocessed_data_cleaned.csv')


df_cleaned = df_cleaned[df_cleaned['country'] != ',']
df_cleaned = df_cleaned[df_cleaned['country'] != '\"n/a\""']
df_cleaned= df_cleaned.drop(columns=['user_id','Unnamed: 0','publisher','location','Language','Summary','year_of_publication','Category','city','state'])



df_cleaned['country']=df_cleaned['country'].str.split(',').str[-1]
df_cleaned['country']= df_cleaned['country'].replace(r' united kingdom',r'united kingdom')
df_cleaned['country']= df_cleaned['country'].replace(r'\"n/a\""',r'unknown')
df_cleaned= df_cleaned.sort_values(by=['country'],ascending=True)
df_cleaned=df_cleaned.reset_index(drop=True)
(df_cleaned['country']=='\"n/a\""').sum()
df_cleaned.head()

,age,isbn,rating,book_title,book_author,country
0,33.0,0140310657,10,The TV Kid (Puffin Books),Betsy Byars,australia
1,14.0,0713100788,9,Discovering Greek Mythology (Woburn Educationa...,P. Kenneth Corsar,australia
2,33.0,0140328262,10,The Paw Thing (Puffin Books),Paul Jennings,australia
3,13.0,0340634448,5,The Cowboy Guide,Wayne Caba,australia
4,18.0,0590454471,2,Picklemania (School Daze),Jerry Spinelli,australia


In [96]:
df_cleaned['Vote-Average'] = df_cleaned['rating'].groupby([df_cleaned['isbn'],df_cleaned['country']]).transform('mean')

df_cleaned.to_csv('test2.csv', index=False)

SyntaxError: invalid syntax (2512353099.py, line 1)

-----------------------------------------------------------
# Content Filtering

In [63]:
# Loading csv with preprocessed data cleaned
# Here add your path to the Preprocessed_data_cleaned.csv file!
# You just need to run the first and this cell from now on everytime you want to run
# the code
df_cleaned = pd.read_csv('/Users/mep/Desktop/mep/CompTools/BookRecommendationSystem/Preprocessed_data_cleaned.csv')

--------------------------------------------------------------------------------
# Collaborative Filtering

In [ ]:
# Loading csv with preprocessed data cleaned
# Here add your path to the Preprocessed_data_cleaned.csv file!
# You just need to run the first and this cell from now on everytime you want to run
# the code
df_cleaned = pd.read_csv('/Users/mep/Desktop/mep/CompTools/BookRecommendationSystem/Preprocessed_data_cleaned.csv')